In [ ]:


import os
import sys

def simulate_turing_machine(binary1, binary2):
    """
    Simulate a Turing machine to perform binary multiplication of two numbers.
    Each step of the tape is logged into a `.dat` file.
    """
    #defining the Turing machine config
    states = {"q0", "q1", "q2", "q3", "q4", "q5", "q6", "q7", "q8", "q9", "q10", "q11", "q12", "q13", "q14", "q15", "q16", "q17", "q18", "q19", "q20", "qhalt"}
    tape_alphabet = {"0", "1", "B", "#", "$", "X", "Y"}
    blank_symbol = "B"
    input_alphabet = {"0", "1"}
    initial_state = "q0"
    halt_state = "qhalt"

    #converts binary strings to integers
    num1 = int(binary1, 2)
    num2 = int(binary2, 2)

    # Initialize the Turing machine tape
    tape = list(binary1 + "#" + binary2 + "$" + "B" * 100)  # Extra blank space for computation
    head_pos = 0  # Initial head position
    state = initial_state
    tape_history = []  # Store tape configurations at each step

    # Function to update the tape and head position based on transition rules
    def transition(state, symbol):
        if state == "q0":
            if symbol == "1":
                return "q1", "X", 1
            elif symbol == "0":
                return "q5", "B", 1
        elif state == "q1":
            if symbol == "1" or symbol == "0":
                return "q1", symbol, 1
            elif symbol == "#":
                return "q2", "#", 1
        elif state == "q2":
            if symbol == "1":
                return "q3", "Y", -1
            elif symbol == "0":
                return "q4", "B", 1
        elif state == "q3":
            if symbol == "X":
                return "q0", "X", 1
            elif symbol == "1" or symbol == "0":
                return "q3", symbol, -1
        elif state == "q4":
            if symbol == "#":
                return "qhalt", "#", 0
        elif state == "q5":
            if symbol == "1" or symbol == "0":
                return "q5", symbol, 1
            elif symbol == "#":
                return "q6", "#", 1
        elif state == "q6":
            if symbol == "1":
                return "q7", "Y", -1
            elif symbol == "0":
                return "q8", "B", 1
        elif state == "q7":
            if symbol == "X":
                return "q5", "X", 1
            elif symbol == "1" or symbol == "0":
                return "q7", symbol, -1
        elif state == "q8":
            if symbol == "#":
                return "qhalt", "#", 0
        return state, symbol, 0 


    max_steps = 10000  #upper limit
    step_count = 0
    while state != halt_state and step_count < max_steps:
    #Turing machine logic
        step_count += 1

    if step_count >= max_steps:
    
        print("Error: Reached max steps. Possible infinite loop.")
    
    if head_pos < 0:
        tape.insert(0, blank_symbol)
        head_pos = 0
    elif head_pos >= len(tape):
        tape.append(blank_symbol)


    #start the Turing machine
    while state != halt_state:
        current_symbol = tape[head_pos]
        new_state, new_symbol, move = transition(state, current_symbol)
        tape[head_pos] = new_symbol
        head_pos += move
        state = new_state

        #records the current state of the tape
        tape_history.append((state, head_pos, ''.join(tape)))

    #generates a filename based on input
    filename = f"binary_mult_{binary1}_x_{binary2}.dat"

    #writes the tape history to a .dat file
    with open(filename, "w") as file:
        for step in tape_history:
            file.write(f"{step[0]} {step[1]} {step[2]}\n")

    print(f"Turing machine simulation complete. Output saved to {filename}.")

    print(f"State: {state}, Head Pos: {head_pos}, Tape: {''.join(tape)}") 

    #checks if file errors occurs
    try:
        with open(filename, "w") as file:
            for step in tape_history:
                file.write(f"{step[0]} {step[1]} {step[2]}\\n")
    except Exception as e:
    
        print(f"Error writing to file: {e}")


simulate_turing_machine("10", "11")


Error: Reached max steps. Possible infinite loop.


In [11]:
#part a attempt 2 

class TuringMachine:
    def __init__(self, tape, states, initial_state, halt_state, blank_symbol, transition_function):
        self.tape = list(tape) + [blank_symbol] * 100  
        self.blank_symbol = blank_symbol
        self.head_position = 0
        self.current_state = initial_state
        self.halt_state = halt_state
        self.transition_function = transition_function
        self.history = []

    def step(self):
        current_symbol = self.tape[self.head_position]
        if (self.current_state, current_symbol) in self.transition_function:
            new_state, new_symbol, direction = self.transition_function[(self.current_state, current_symbol)]
            self.tape[self.head_position] = new_symbol
            self.current_state = new_state
            self.head_position += 1 if direction == 'R' else -1

            #extends the tape if the head moves beyond the current bounds
            if self.head_position < 0:
                self.tape.insert(0, self.blank_symbol)
                self.head_position = 0
            elif self.head_position >= len(self.tape):
                self.tape.append(self.blank_symbol)

            #records the state of the tape
            self.history.append((''.join(self.tape), self.current_state, self.head_position))
        else:
            raise Exception(f"No transition defined for state {self.current_state} and symbol {current_symbol}")

    def run(self):
        while self.current_state != self.halt_state:
            self.step()

    def save_history(self, filename):
        with open(filename, "w") as file:
            for step in self.history:
                tape, state, head_position = step
                file.write(f"{tape} {state} {head_position}\n")


#defines the transition function 

transition_function = {
    ("q0", "1"): ("q1", "X", "R"),
    ("q0", "0"): ("q5", "B", "R"),
    ("q1", "1"): ("q1", "1", "R"),
    ("q1", "0"): ("q1", "0", "R"),
    ("q1", "#"): ("q2", "#", "R"),
    ("q2", "1"): ("q3", "Y", "L"),
    ("q2", "0"): ("q4", "B", "R"),
    ("q3", "X"): ("q0", "X", "R"),
    ("q3", "1"): ("q3", "1", "L"),
    ("q3", "0"): ("q3", "0", "L"),
    ("q3", "#"): ("q4", "#", "R"),
    ("q4", "#"): ("qhalt", "#", "N"),
    ("q4", "Y"): ("q4", "Y", "R"),
    ("q4", "1"): ("q4", "1", "R"),
    ("q4", "0"): ("q4", "0", "R"),
    ("q4", "$"): ("qhalt", "$", "N"), 
    ("q5", "1"): ("q5", "1", "R"),
    ("q5", "0"): ("q5", "0", "R"),
    ("q5", "#"): ("q6", "#", "R"),
    ("q6", "1"): ("q7", "Y", "L"),
    ("q6", "0"): ("q8", "B", "R"),
    ("q7", "X"): ("q5", "X", "R"),
    ("q7", "1"): ("q7", "1", "L"),
    ("q7", "0"): ("q7", "0", "L"),
    ("q8", "#"): ("qhalt", "#", "N"),
}






#Input tape
binary1 = "101001010111"
binary2 = "101000101"
tape = binary1 + "#" + binary2 + "$"

#initializes the Turing machine
machine = TuringMachine(
    tape=tape,
    states={"q0", "q1", "q2", "q3", "q4", "q5", "q6", "q7", "q8", "qhalt"},
    initial_state="q0",
    halt_state="qhalt",
    blank_symbol="B",
    transition_function=transition_function
)


try:
    machine.run()
except Exception as e:
    print(f"Error: {e}")

# Save the history to a .dat file
filename = f"turing_mult_{binary1}_x_{binary2}.dat"
machine.save_history(filename)
print(f"Turing machine simulation complete. Output saved to {filename}.")


Turing machine simulation complete. Output saved to turing_mult_101001010111_x_101000101.dat.


In [12]:
#[2,3] 

class TuringMachine:
    def __init__(self, tape, states, initial_state, halt_state, blank_symbol, transition_function):
        self.tape = list(tape) + [blank_symbol] * 100  
        self.blank_symbol = blank_symbol
        self.head_position = 0
        self.current_state = initial_state
        self.halt_state = halt_state
        self.transition_function = transition_function
        self.history = []

    def step(self):
        current_symbol = self.tape[self.head_position]
        if (self.current_state, current_symbol) in self.transition_function:
            new_state, new_symbol, direction = self.transition_function[(self.current_state, current_symbol)]
            self.tape[self.head_position] = new_symbol
            self.current_state = new_state
            self.head_position += 1 if direction == 'R' else -1

            #extends the tape if the head moves beyond the current bounds
            if self.head_position < 0:
                self.tape.insert(0, self.blank_symbol)
                self.head_position = 0
            elif self.head_position >= len(self.tape):
                self.tape.append(self.blank_symbol)

            #records the state of the tape
            self.history.append((''.join(self.tape), self.current_state, self.head_position))
        else:
            raise Exception(f"No transition defined for state {self.current_state} and symbol {current_symbol}")

    def run(self):
        while self.current_state != self.halt_state:
            self.step()

    def save_history(self, filename):
        with open(filename, "w") as file:
            for step in self.history:
                tape, state, head_position = step
                file.write(f"{tape} {state} {head_position}\n")


#defines the transition function 

transition_function = {
    ("q0", "1"): ("q1", "X", "R"),
    ("q0", "0"): ("q5", "B", "R"),
    ("q1", "1"): ("q1", "1", "R"),
    ("q1", "0"): ("q1", "0", "R"),
    ("q1", "#"): ("q2", "#", "R"),
    ("q2", "1"): ("q3", "Y", "L"),
    ("q2", "0"): ("q4", "B", "R"),
    ("q3", "X"): ("q0", "X", "R"),
    ("q3", "1"): ("q3", "1", "L"),
    ("q3", "0"): ("q3", "0", "L"),
    ("q3", "#"): ("q4", "#", "R"),
    ("q4", "#"): ("qhalt", "#", "N"),
    ("q4", "Y"): ("q4", "Y", "R"),
    ("q4", "1"): ("q4", "1", "R"),
    ("q4", "0"): ("q4", "0", "R"),
    ("q4", "$"): ("qhalt", "$", "N"), 
    ("q5", "1"): ("q5", "1", "R"),
    ("q5", "0"): ("q5", "0", "R"),
    ("q5", "#"): ("q6", "#", "R"),
    ("q6", "1"): ("q7", "Y", "L"),
    ("q6", "0"): ("q8", "B", "R"),
    ("q7", "X"): ("q5", "X", "R"),
    ("q7", "1"): ("q7", "1", "L"),
    ("q7", "0"): ("q7", "0", "L"),
    ("q8", "#"): ("qhalt", "#", "N"),
}






#Input tape
binary1 = "101"
binary2 = "101001"
tape = binary1 + "#" + binary2 + "$"

#initializes the Turing machine
machine = TuringMachine(
    tape=tape,
    states={"q0", "q1", "q2", "q3", "q4", "q5", "q6", "q7", "q8", "qhalt"},
    initial_state="q0",
    halt_state="qhalt",
    blank_symbol="B",
    transition_function=transition_function
)


try:
    machine.run()
except Exception as e:
    print(f"Error: {e}")

# Save the history to a .dat file
filename = f"turing_mult_{binary1}_x_{binary2}.dat"
machine.save_history(filename)
print(f"Turing machine simulation complete. Output saved to {filename}.")


Turing machine simulation complete. Output saved to turing_mult_101_x_101001.dat.
